In [ ]:
import glob
import pandas
import matplotlib.pyplot as plt
import seaborn
import json
import numpy as np

In [ ]:
outdir = "../outputs/plots/"

In [ ]:
!ls /local/joosep/ml-tau-en-reg/results

In [ ]:
!ls /local/joosep/ml-tau-en-reg/results/240608_PT_num_layers_4_all/240608_PT_num_layers_4_all_v*/*/*/history.json

In [ ]:
losses = {
    "jet_regression": {"LorentzNet": [], "ParticleTransformer": [], "SimpleDNN": []},
    "dm_multiclass": {"LorentzNet": [], "ParticleTransformer": [], "SimpleDNN": []},
}

for path in sorted(glob.glob("/local/joosep/ml-tau-en-reg/results/240608_PT_num_layers_4_all/*/*/*/history.json")):
    spl = path.split("/")
    model = spl[-2]
    target = spl[-3]
    print(model, target)
    losses[target][model].append(json.load(open(path)))

In [ ]:
losses_avg = {}
losses_std = {}
for target in ["dm_multiclass", "jet_regression"]:
    losses_avg[target] = {}
    losses_std[target] = {}
    for model in ["LorentzNet", "ParticleTransformer", "SimpleDNN"]:
        losses_avg[target][model] = {}
        losses_std[target][model] = {}
        for loss_name in ["losses_train", "losses_validation"]:
            loss = np.array([l[loss_name] for l in losses[target][model]])
            losses_avg[target][model][loss_name] = np.mean(loss, axis=0)
            losses_std[target][model][loss_name] = np.std(loss, axis=0)

In [ ]:
training_type = "dm_multiclass"
training_title = "Decay mode classification"

fig = plt.figure()
ax = plt.axes()

for model in ["SimpleDNN", "LorentzNet", "ParticleTransformer"]: 
    c = plt.plot(losses_avg[training_type][model]["losses_train"], ls="--", lw=1)[0]
    plt.plot(losses_avg[training_type][model]["losses_validation"], label=model, color=c.get_color())
    plt.fill_between(
        range(100),
        losses_avg[training_type][model]["losses_train"] - losses_std[training_type][model]["losses_train"],
        losses_avg[training_type][model]["losses_train"] + losses_std[training_type][model]["losses_train"],
        color=c.get_color(),
        alpha=0.5
    )
    
    plt.fill_between(
        range(100),
        losses_avg[training_type][model]["losses_validation"] - losses_std[training_type][model]["losses_validation"],
        losses_avg[training_type][model]["losses_validation"] + losses_std[training_type][model]["losses_validation"],
        color=c.get_color(),
        alpha=0.5
    )
    
plt.ylim(0.2, 0.4)
leg1 = plt.legend(loc=1, frameon=False)
ax.add_artist(leg1)
plt.ylabel("Loss")
plt.xlabel("Training epoch")
plt.title(training_title)

import matplotlib.lines as mlines
handles = [mlines.Line2D([], [], color="black", ls="--"), mlines.Line2D([], [], color="black", ls="-")]
ax.legend(handles, ['training', 'validation'], loc=3, frameon=False)
plt.savefig(outdir + "/loss_{}.pdf".format(training_type))

In [ ]:
training_type = "jet_regression"
training_title = "Momentum regression"

fig = plt.figure()
ax = plt.axes()

for model in ["SimpleDNN", "LorentzNet", "ParticleTransformer"]: 
    c = plt.plot(losses_avg[training_type][model]["losses_train"], ls="--", lw=1)[0]
    plt.plot(losses_avg[training_type][model]["losses_validation"], label=model, color=c.get_color())
    plt.fill_between(
        range(100),
        losses_avg[training_type][model]["losses_train"] - losses_std[training_type][model]["losses_train"],
        losses_avg[training_type][model]["losses_train"] + losses_std[training_type][model]["losses_train"],
        color=c.get_color(),
        alpha=0.5
    )
    
    plt.fill_between(
        range(100),
        losses_avg[training_type][model]["losses_validation"] - losses_std[training_type][model]["losses_validation"],
        losses_avg[training_type][model]["losses_validation"] + losses_std[training_type][model]["losses_validation"],
        color=c.get_color(),
        alpha=0.5
    )
    
plt.ylim(0.0, 0.02)
leg1 = plt.legend(loc=1, frameon=False)
ax.add_artist(leg1)
plt.ylabel("Loss")
plt.xlabel("Training epoch")
plt.title(training_title)

import matplotlib.lines as mlines
handles = [mlines.Line2D([], [], color="black", ls="--"), mlines.Line2D([], [], color="black", ls="-")]
ax.legend(handles, ['training', 'validation'], loc=3, frameon=False)
plt.savefig(outdir + "/loss_{}.pdf".format(training_type))

In [ ]:
training_type = "jet_regression"

def extract_losses(training_type):
    vals_loss = []
    vals_loss_std = []
    
    for model in ["SimpleDNN", "LorentzNet", "ParticleTransformer"]:
        bestidx = np.argmin(losses_avg["jet_regression"][model]["losses_validation"])
        best_loss = losses_avg[training_type][model]["losses_validation"][bestidx]
        best_loss_std = losses_std[training_type][model]["losses_validation"][bestidx]
    
        vals_loss.append(best_loss)
        vals_loss_std.append(best_loss_std)
    
    vals_loss = np.array(vals_loss)
    vals_loss_std = np.array(vals_loss_std)

    return vals_loss, vals_loss_std

vals_loss_jr, vals_loss_std_jr = extract_losses("jet_regression")
vals_loss_dm, vals_loss_std_dm = extract_losses("dm_multiclass")

xticks = np.arange(3)
plt.bar(xticks-0.2, vals_loss_jr/vals_loss_jr[0], yerr=vals_loss_std_jr/vals_loss_jr[0], width=0.4, label="Momentum regression")
plt.bar(xticks+0.2, vals_loss_dm/vals_loss_dm[0], yerr=vals_loss_std_dm/vals_loss_dm[0], width=0.4, label="Decay mode classification")

plt.legend(frameon=False)
plt.xticks(xticks, ["DeepSet", "LorentzNet", "ParticleTransformer"])
plt.ylabel("Best validation loss, relative to DeepSet")
plt.ylim(0,1.2)
plt.axhline(1.0, color="black", ls="--", lw=0.5)